In [1]:
import requests
import pandas as pd
import time

from datetime import date
from datetime import timedelta

pd.set_option("display.max_columns", None)

In [2]:
from functions import make_request
from functions import get_recent_games

In [3]:
teams = make_request("teams", record_path="data")

team_codes = teams[["id", "name"]]

team_codes

,id,name
0,1,Hawks
1,2,Celtics
2,3,Nets
3,4,Hornets
4,5,Bulls
5,6,Cavaliers
6,7,Mavericks
7,8,Nuggets
8,9,Pistons
9,10,Warriors


In [4]:
# Get a pandas dataframe of the 20 most recent games for the following team
home_team_id = 1

today = date.today()                                                           # Get today
today = f"{today.year}-{today.month}-{today.day}"                              # Convert to format yyyy-mm-dd
one_year_ago = date.today() - timedelta(days=365)                              # Get last-year-today
one_year_ago = f"{one_year_ago.year}-{one_year_ago.month}-{one_year_ago.day}"  # convert to format yyyy-mm-dd

recent_games = pd.DataFrame()
res = make_request("games", record_path="data", params={"end_date": today,
                                                        "start_date": one_year_ago,
                                                        "team_ids[]": [home_team_id],
                                                        "page": "1",
                                                       "per_page": "100"})

res = res.sort_values("date", ascending=False)
res = res[res["home_team.id"].eq(home_team_id)]

recent_games = recent_games.append(res)
recent_games = recent_games.head(20)

In [5]:
recent_games

,id,date,home_team_score,period,postseason,season,status,time,visitor_team_score,home_team.id,home_team.abbreviation,home_team.city,home_team.conference,home_team.division,home_team.full_name,home_team.name,visitor_team.id,visitor_team.abbreviation,visitor_team.city,visitor_team.conference,visitor_team.division,visitor_team.full_name,visitor_team.name
46,473663,2021-11-22T00:00:00.000Z,113,4,False,2021,Final,,101,1,ATL,Atlanta,East,Southeast,Atlanta Hawks,Hawks,21,OKC,Oklahoma City,West,Northwest,Oklahoma City Thunder,Thunder
37,473652,2021-11-20T00:00:00.000Z,115,4,False,2021,Final,,105,1,ATL,Atlanta,East,Southeast,Atlanta Hawks,Hawks,4,CHA,Charlotte,East,Southeast,Charlotte Hornets,Hornets
56,473621,2021-11-17T00:00:00.000Z,110,4,False,2021,Final,,99,1,ATL,Atlanta,East,Southeast,Atlanta Hawks,Hawks,2,BOS,Boston,East,Atlantic,Boston Celtics,Celtics
43,473607,2021-11-15T00:00:00.000Z,129,4,False,2021,Final,,111,1,ATL,Atlanta,East,Southeast,Atlanta Hawks,Hawks,22,ORL,Orlando,East,Southeast,Orlando Magic,Magic
55,473603,2021-11-14T00:00:00.000Z,120,4,False,2021,Final,,100,1,ATL,Atlanta,East,Southeast,Atlanta Hawks,Hawks,17,MIL,Milwaukee,East,Central,Milwaukee Bucks,Bucks
52,473528,2021-11-04T00:00:00.000Z,98,4,False,2021,Final,,116,1,ATL,Atlanta,East,Southeast,Atlanta Hawks,Hawks,29,UTA,Utah,West,Northwest,Utah Jazz,Jazz
47,473505,2021-11-01T00:00:00.000Z,118,4,False,2021,Final,,111,1,ATL,Atlanta,East,Southeast,Atlanta Hawks,Hawks,30,WAS,Washington,East,Southeast,Washington Wizards,Wizards
44,473454,2021-10-25T00:00:00.000Z,122,4,False,2021,Final,,104,1,ATL,Atlanta,East,Southeast,Atlanta Hawks,Hawks,9,DET,Detroit,East,Central,Detroit Pistons,Pistons
35,473424,2021-10-21T00:00:00.000Z,113,4,False,2021,Final,,87,1,ATL,Atlanta,East,Southeast,Atlanta Hawks,Hawks,7,DAL,Dallas,West,Southwest,Dallas Mavericks,Mavericks
2,459132,2021-08-12T00:00:00.000Z,88,5,False,2021,Final,,96,1,ATL,Atlanta,East,Southeast,Atlanta Hawks,Hawks,23,PHI,Philadelphia,East,Atlantic,Philadelphia 76ers,76ers


In [6]:
game_ids_home, game_ids_away = get_recent_games(1,2)

In [7]:
game_ids_home

[473663,
 473652,
 473621,
 473607,
 473603,
 473528,
 473505,
 473454,
 473424,
 459132,
 451506,
 446517,
 445510,
 444934,
 442056,
 440182,
 438743,
 430992,
 429183,
 264794]

In [8]:
def get_stats(game_ids_home, game_ids_away):
    
    def format_params(game_ids):
        """ 
        Format query paramaters in a format the balldontlie API accepts
        e.g. ?game_ids[]=345686&game_ids[]=234356&gameids[]=3456356...
        """
        params = "game_ids[] " * len(game_ids_home)
        params = list(zip(params.split(" "), game_ids_home))
        params.append(("per_page", 100))
        return params
    
    # Get pandas dataframe of home team stats
    params_home = format_params(game_ids_home)                                 # Get param list
    stats_home = make_request("stats", record_path="data", params=params_home) # Make request with said param list
    stats_home = stats_home[stats_home["team.id"].eq(stats_home["game.home_team_id"])]  # Filter for stats of players that played for the home team
    
    
    # Get pandas dataframe of away team stats
    params_away = format_params(game_ids_away)
    stats_away = make_request("stats", record_path="data", params=params_away)
    
    return stats_home, stats_away

In [10]:
stats_home, stats_away = get_stats(game_ids_home, game_ids_away)

In [24]:
stats_home[stats_home["team.id"].eq(stats_home["game.home_team_id"])].groupby("game.id")

,id,ast,blk,dreb,fg3_pct,fg3a,fg3m,fg_pct,fga,fgm,ft_pct,fta,ftm,min,oreb,pf,pts,reb,stl,turnover,game.id,game.date,game.home_team_id,game.home_team_score,game.period,game.postseason,game.season,game.status,game.time,game.visitor_team_id,game.visitor_team_score,player.id,player.first_name,player.height_feet,player.height_inches,player.last_name,player.position,player.team_id,player.weight_pounds,team.id,team.abbreviation,team.city,team.conference,team.division,team.full_name,team.name
0,6528101,6,0,2,0.0,0,0,66.7,6,4,100.0,3,3,16:33,0,1,11,2,0,4,264794,2021-05-16T00:00:00.000Z,1,124,4,False,2020,Final,,11,95,481,Lou,6.0,1.0,Williams,G,1,175.0,1,ATL,Atlanta,East,Southeast,Atlanta Hawks,Hawks
3,6528100,1,0,2,0.0,0,0,57.1,7,4,0.0,0,0,23:33,0,3,8,2,2,2,264794,2021-05-16T00:00:00.000Z,1,124,4,False,2020,Final,,11,95,426,Tony,6.0,7.0,Snell,G,25,213.0,1,ATL,Atlanta,East,Southeast,Atlanta Hawks,Hawks
4,6528093,4,0,4,33.3,6,2,28.6,7,2,0.0,0,0,35:47,1,2,6,5,2,1,264794,2021-05-16T00:00:00.000Z,1,124,4,False,2020,Final,,11,95,212,Solomon,6.0,7.0,Hill,F,1,225.0,1,ATL,Atlanta,East,Southeast,Atlanta Hawks,Hawks
6,6528095,0,0,0,0.0,0,0,0.0,0,0,0.0,0,0,,0,0,0,0,0,0,264794,2021-05-16T00:00:00.000Z,1,124,4,False,2020,Final,,11,95,139,Kris,6.0,4.0,Dunn,G,15,205.0,1,ATL,Atlanta,East,Southeast,Atlanta Hawks,Hawks
8,6528091,0,1,4,100.0,1,1,71.4,7,5,100.0,4,4,16:20,0,1,15,4,0,0,264794,2021-05-16T00:00:00.000Z,1,124,4,False,2020,Final,,11,95,101,John,6.0,10.0,Collins,F-C,1,235.0,1,ATL,Atlanta,East,Southeast,Atlanta Hawks,Hawks
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50,8265986,6,0,3,55.6,9,5,45.0,20,9,100.0,7,7,31:55,1,2,30,4,1,2,473663,2021-11-22T00:00:00.000Z,1,113,4,False,2021,Final,,21,101,490,Trae,6.0,2.0,Young,G,1,180.0,1,ATL,Atlanta,East,Southeast,Atlanta Hawks,Hawks
51,8265994,1,0,2,25.0,4,1,36.4,11,4,0.0,0,0,22:59,1,1,9,3,1,0,473663,2021-11-22T00:00:00.000Z,1,113,4,False,2021,Final,,21,101,666860,Cameron,NaN,NaN,Reddish,F-G,1,NaN,1,ATL,Atlanta,East,Southeast,Atlanta Hawks,Hawks
57,8265993,0,0,0,0.0,0,0,0.0,0,0,0.0,0,0,1:27,0,0,0,0,0,0,473663,2021-11-22T00:00:00.000Z,1,113,4,False,2021,Final,,21,101,3547289,Skylar,NaN,NaN,Mays,G,1,NaN,1,ATL,Atlanta,East,Southeast,Atlanta Hawks,Hawks
60,8265987,0,0,0,0.0,0,0,0.0,0,0,0.0,0,0,1:27,0,0,0,0,0,0,473663,2021-11-22T00:00:00.000Z,1,113,4,False,2021,Final,,21,101,17896029,Sharife,NaN,NaN,Cooper,G,1,NaN,1,ATL,Atlanta,East,Southeast,Atlanta Hawks,Hawks


In [21]:
stats_home["team.id"].value_counts()

1     292
23     59
17     57
20     30
2      25
7      15
9      13
30     13
29     13
11     12
22     12
21     12
4      11
Name: team.id, dtype: int64

In [ ]:
stats_away